# **저시력자를 위한 원화 화폐 분류**
---
- UltraLytics YOLO v5 모델 사용
---

## 0.미션
---
- **과제 수행 목표**
    - Object Detection
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리** 필요
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있음
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제
---
- [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
---

In [4]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [5]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [6]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조
        - Image와 Label을 구분
---
- [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
---

In [7]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [8]:
import glob, shutil

In [9]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- Training과 Validation은 8:2로 분리
- 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣기
    - /dataset/images/train, /dataset/labels/train
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
---

In [ ]:
# 3. 데이터를 Training set | Validation set으로 분할하세요.
# 이미지 및 JSON 파일 가져오기
image_files = glob.glob(os.path.join(data_path, '*/*.jpg'))
json_files = glob.glob(os.path.join(data_path, '*/*.json'))
print(len(image_files), len(json_files))
image_files.sort()
json_files.sort()

In [ ]:
print(image_files[:5])
print(json_files[:5])

In [ ]:
from sklearn.model_selection import train_test_split

train_img, val_img, train_json, val_json = train_test_split(image_files, json_files, test_size=0.2, shuffle=True, random_state=2023)
print(train_img[:5])
print(train_json[:5])

In [13]:
def move_file(target, move_list):
    for path in move_list:
        file_name = os.path.basename(path)
        target_path = os.path.join(target, file_name)
        shutil.copy(path, target_path)

move_file('/content/Dataset/images/train', train_img)
move_file('/content/Dataset/images/val', val_img)
move_file('/content/Dataset/labels/train', train_json)
move_file('/content/Dataset/labels/val', val_json)

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분
        - 화폐의 앞뒷면 구분을 없애기
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경
        - 사용되는 이미지는 원본에서 1/5로 축소
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여야 하지만 yolov5는 이미지 위치 정보가 0에서 1로 스케일링 되어야 함
        - 그냥 너비와 높이로 나눠서 크기 자체를 스케일링해서 제출
    - 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장
        - YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [14]:
import os, json
import numpy as np
import math

In [15]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [ ]:
train_j = glob.glob(os.path.join(json_path, 'train/*.json'))
val_j = glob.glob(os.path.join(json_path, 'val/*.json'))
print(len(train_j), len(val_j))

class_mapping ={'Ten_front':0,
                'Ten_back':0,
                'Fifty_front':1,
                'Fifty_back':1,
                'Hundred_front':2,
                'Hundred_back':2,
                'Five_Hundred_front':3,
                'Five_Hundred_back':3,
                'Thousand_front':4,
                'Thousand_back':4,
                'Five_Thousand_front':5,
                'Five_Thousand_back':5,
                'Ten_Thousand_front':6,
                'Ten_Thousand_back':6,
                'Fifty_Thousand_front':7,
                'Fifty_Thousand_back':7
                }

check_list = []

# train json 나누기
for path in train_j:
    with open(path, 'r') as json_file:
        d = json.load(json_file)

    json_file_name = os.path.basename(path)
    txt_file_name = os.path.splitext(json_file_name)[0] + '.txt' # splitext : 파일명과 확장자로 split
    output_txt_file = os.path.join(os.path.dirname(path), txt_file_name)

    image_width = d['imageWidth']
    image_height = d['imageHeight']

    with open(output_txt_file, 'w') as output_file:
        for shape in d['shapes']:
            # 클래스 정보 추출 및 클래스 매핑 적용
            original_class = shape['label']
            mapped_class = class_mapping.get(original_class, original_class)

            # 잘 받았는지 확인용
            check_list.append(mapped_class)

            # 위치 정보
            x1 = shape['points'][0][0]/image_width
            y1 = shape['points'][0][1]/image_height
            x2 = shape['points'][1][0]/image_width
            y2 = shape['points'][1][1]/image_height
            #print(x1, y1, x2, y2)
            width = abs(x2 - x1)
            height = abs(y1 - y2)
            x_center = min(x1, x2) + (width / 2)
            y_center = min(y1, y2) + (height / 2)
            #print(image_width, image_height, width, height, x_center, y_center)
            # label, x-center, y-center, width, height

            # label, x-center, y-center, width, height
            yolo_label = f"{mapped_class} {x_center} {y_center} {width} {height}\n"

            output_file.write(yolo_label)

print(set(check_list))
print(len(check_list))

In [ ]:
check_list = []

for path in val_j:
    with open(path, 'r') as json_file:
        d = json.load(json_file)

    json_file_name = os.path.basename(path)
    txt_file_name = os.path.splitext(json_file_name)[0] + '.txt' # splitext : 파일명과 확장자로 split
    output_txt_file = os.path.join(os.path.dirname(path), txt_file_name)

    image_width = d['imageWidth']
    image_height = d['imageHeight']
    with open(output_txt_file, 'w') as output_file:
        for shape in d['shapes']:
            # 클래스 정보 추출 및 클래스 매핑 적용
            original_class = shape['label']
            mapped_class = class_mapping.get(original_class, original_class)

            # 잘 받았는지 확인용
            check_list.append(mapped_class)

            # 위치 정보
            x1 = shape['points'][0][0]/image_width
            y1 = shape['points'][0][1]/image_height
            x2 = shape['points'][1][0]/image_width
            y2 = shape['points'][1][1]/image_height
            #print(x1, y1, x2, y2)
            width = abs(x2 - x1)
            height = abs(y1 - y2)
            x_center = min(x1, x2) + (width / 2)
            y_center = min(y1, y2) + (height / 2)
            #print(image_width, image_height, width, height, x_center, y_center)
            # label, x-center, y-center, width, height
            yolo_label = f"{mapped_class} {x_center} {y_center} {width} {height}\n"

            output_file.write(yolo_label)

print(set(check_list))
print(len(check_list))

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
---

In [18]:
import yaml

In [19]:
won_dict = {0:10, 1:50, 2:100, 3:500, 4:1000, 5:5000, 6:10000, 7:50000}

In [20]:
class_names = set(class_mapping.values())
num_classes = len(class_names)

dataset_info_dict = {
    "nc": num_classes,
}

dataset_info_dict['names'] = {i: won_dict[i] for i in range(num_classes)}
dataset_info_dict['train'] = '/content/Dataset/images/train'
dataset_info_dict['val'] = "/content/Dataset/images/val"

# YAML 파일로 저장
output_yaml_file = "/content/Dataset/dataset_info.yaml"

with open(output_yaml_file, 'w') as yaml_file:
    yaml.dump(dataset_info_dict, yaml_file, default_flow_style=False, default_style='')


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

### (2) 가중치 파일 다운 및 train.py


In [ ]:
# Fine-tuning을 시작합니다.
!python yolov5/train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/Dataset/dataset_info.yaml \
  --weights yolov5s.pt \
  --project /content/yolov5_training \
  --name yolov5s_results \
  --exist-ok \
  --device 0

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

현재 모델을 돌린 후 나온 weight를 드라이브에 따로 저장해 둔 상태
+ 모델을 다시 돌릴때는 제 드라이브에 있는 가중치로 돌린 상태
    + 런타임이 끝나면 가중치가 사라져서 미리 백업을 해놨습니다.

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/m1.jpeg

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_video.mp4

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best.pt --conf-thres 0.2 --iou-thres 0.25 --source https://blog.kakaocdn.net/dn/CaV04/btqYZ6be0jb/3Ny1H2EGEShy92s6yqiFN1/img.jpg

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_overlap1.jpeg

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_overlap2.jpg

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp/m1.jpeg"

# 이미지 열기
Image.open(image_path)

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp4/img.jpg"

# 이미지 열기
Image.open(image_path)


In [ ]:
import shutil

source_path = "/content/yolov5_training/yolov5s_results/weights/best.pt"
destination_path = "/content/drive/MyDrive/KT/miniproject3/best.pt"

shutil.copy(source_path, destination_path)

## 5.하이퍼 파라미터 바꿔보기 :  hyp.scratch-low.yaml
+ 하이퍼파라미터 default는 'hyp.scratch-low.yaml'
+ 해당 파일에서 몇 부분을 수정해서 설정
```
def parse_opt(known=False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default=ROOT / 'yolov5s.pt', help='initial weights path')
    parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--hyp', type=str, default=ROOT / 'data/hyps/hyp.scratch-low.yaml', help='hyperparameters path')
```

+ mixup: 0.0  # image mixup (probability)
    + 겹친 이미지에 대한 판단이 힘들어 보여서 1.0으로 줘서 해보기
+ degrees: 0.0  # image rotation (+/- deg)
    + 동영상 촬영으로 이미지 촬영시 0도, 90도, 180도, 270도는 잘 인식하지만 그 사이각이 인식이 잘 안되는 문제점 발생
    + 90도의 rotation을 줘서 학습시키기

In [ ]:
!python yolov5/train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --hyp /content/drive/MyDrive/KT/miniproject3/hyp.scratch-custom.yaml \
  --data /content/Dataset/dataset_info.yaml \
  --weights yolov5s.pt \
  --project /content/yolov5_training_custom \
  --name yolov5s_results \
  --exist-ok \
  --device 0

In [ ]:
import shutil

source_path = "/content/yolov5_training/yolov5s_results/weights/best.pt"
destination_path = "/content/drive/MyDrive/KT/miniproject3/best_rotation3.pt"

shutil.copy(source_path, destination_path)

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best_rotation3.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/m1.jpeg

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp4/m1.jpeg"

# 이미지 열기
Image.open(image_path)

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best_rotation3.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_video.mp4

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best_rotation3.pt --conf-thres 0.2 --iou-thres 0.25 --source https://blog.kakaocdn.net/dn/CaV04/btqYZ6be0jb/3Ny1H2EGEShy92s6yqiFN1/img.jpg

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp6/img.jpg"

# 이미지 열기
Image.open(image_path)


In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best_rotation3.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_overlap1.jpeg

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp7/money_overlap1.jpeg"

# 이미지 열기
Image.open(image_path)

In [ ]:
!python ./yolov5/detect.py --weights /content/drive/MyDrive/KT/miniproject3/best_rotation3.pt --conf-thres 0.2 --iou-thres 0.25 --source /content/drive/MyDrive/KT/miniproject3/money_overlap2.jpg

In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = "/content/yolov5/runs/detect/exp8/money_overlap2.jpg"

# 이미지 열기
Image.open(image_path)